In [ ]:
import pygplates
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

%matplotlib inline


#########################################
###### Make a loop
Data = pd.read_csv('/Users/wenchaocao/Research/5_ConsistencyRatios_Fossils_Paleogeog/ReconPaleogeog_EB2015v1_AfterModifiedBasedOnFossils_410_2Ma/ReconPaleogeog_EB2015v1_AfterModifiedOnFossils_TestByFossils_410_2Ma.csv', delimiter=',')  
reconstruction_time_list = Data['Time intervals']
FigNum_list = np.arange(64,17,-2)
FromAge_list = Data['From_age']
ToAge_list = Data['To_age']

recon_time = []
FigNum = []
age_max= []
age_min= []
anchor_plate= 0
#Rotation_time = recon_time

for i,recon_time,FigNum,age_max,age_min in zip(np.arange(1,25,1),reconstruction_time_list,FigNum_list,FromAge_list,ToAge_list):
    print recon_time,FigNum,age_max, age_min
#########################################


    pbdb = pd.read_csv('/Users/wenchaocao/Research/6_CookieCutting_Corrections/Fossil_Collections_Genus_PBDB_2500_0Ma/2500Ma-collections.csv')

    #extract subsets
    pbdb_subset = pbdb[(pbdb.ma_max<=age_max) & (pbdb.ma_min>=age_min)] 

    # input
    base_directory = '/Users/wenchaocao/Research/6_CookieCutting_Corrections/'+str(recon_time)+'/7_2_Reconstructed_Paleogeog_Matthews2016_'+str(recon_time)+'Ma_GapsFixed/'
    ShallowMarine_Filename = '%s/sm_fig%d_%d_%d_reconstructed_%0.2fMa.shp' % (base_directory,FigNum,age_max,age_min,recon_time)
    Landmass_Filename = '%s/lm_fig%d_%d_%d_reconstructed_%0.2fMa.shp' % (base_directory,FigNum,age_max,age_min,recon_time)
    Mountain_Filename = '%s/m_fig%d_%d_%d_reconstructed_%0.2fMa.shp' % (base_directory,FigNum,age_max,age_min,recon_time)
    Icesheet_Filename = '%s/i_fig%d_%d_%d_reconstructed_%0.2fMa.shp' % (base_directory,FigNum,age_max,age_min,recon_time)

    # rotations:
    rotation_filename_for_fossils = ['/Users/wenchaocao/Research/6_CookieCutting_Corrections/Matthews++_410-0Ma_latest/Global_EB_250-0Ma_GK07_Matthews++.rot',
                                    '/Users/wenchaocao/Research/6_CookieCutting_Corrections/Matthews++_410-0Ma_latest/Global_EB_410-250Ma_GK07_Matthews++.rot']

    static_polygons = '/Users/wenchaocao/Research/6_CookieCutting_Corrections/Matthews++_410-0Ma_latest/StaticPolys/PresentDay_StaticPlatePolygons_Matthews++.shp'

    rotation_filename_for_paleogeog = '/Users/wenchaocao/Desktop/New_Rotation.rot'
    #######################################

    rotation_model = pygplates.RotationModel(rotation_filename_for_fossils)

    point_features = []

    for index,row in pbdb_subset.iterrows():
        point = pygplates.PointOnSphere(float(row.latdec),float(row.lngdec))
        #print row.LONG
        point_feature = pygplates.Feature()
        point_feature.set_geometry(point)
        point_feature.set_shapefile_attribute('Environ',str(row.environment))
        point_feature.set_shapefile_attribute('lithology1',str(row.lithology1))
        point_feature.set_shapefile_attribute('lithology2',str(row.lithology2))
        point_feature.set_shapefile_attribute('coll_no',str(row.collection_no))
        point_feature.set_shapefile_attribute('coll_auth',str(row['collection.authorizer']))
        point_feature.set_shapefile_attribute('coll_refno',str(row['collection.reference_no']))
        point_feature.set_shapefile_attribute('ma_max',row.ma_max)
        point_feature.set_shapefile_attribute('ma_min',row.ma_min)
        point_feature.set_shapefile_attribute('period',str(row.period))
        point_feature.set_shapefile_attribute('epoch',str(row.epoch))
        point_feature.set_shapefile_attribute('stage',str(row.stage))
        point_feature.set_shapefile_attribute('GolEnv','Unknown')
        point_feature.set_shapefile_attribute('pbdbEnv','Unknown')
        point_feature.set_shapefile_attribute('Match',0)
        point_feature.set_shapefile_attribute('FromAge',age_max) #added on 6/12/2016
        point_feature.set_shapefile_attribute('ToAge',age_min) #added on 6/12/2016
        point_feature.set_valid_time(age_max,age_min)
        point_features.append(point_feature)

    print len(point_features)

    # The partition points function can then be used as before
    partitioned_point_features = pygplates.partition_into_plates(static_polygons,
                                                           rotation_model,
                                                           point_features) 

    # If partitioned feature has plate id = 0, we throw it away 
    partitioned_point_features_not_zero = []
    partitioned_point_features_zero = []
    for point_feature in partitioned_point_features:
        if point_feature.get_reconstruction_plate_id() != 0:
            partitioned_point_features_not_zero.append(point_feature)
        else:
            partitioned_point_features_zero.append(point_feature)

    # Create a reconstructed version of the fossil features 

    reconstructed_point_features = []
    pygplates.reconstruct(partitioned_point_features_not_zero,
                          rotation_model,
                          reconstructed_point_features,
                          recon_time)
    #print len(partitioned_point_features_not_zero)
    #print len(partitioned_point_features_zero)

    ###################################################
    # This cell contains the mapping between paleo-environments descriptions in PBDB and our own system
    ##### List and filename for Shallow Marine

    shallow_marine_list = ['marine indet.',
                'carbonate indet.',
                'peritidal',
                'shallow subtidal indet.',
                'open shallow subtidal',
                'lagoonal/restricted shallow subtidal',
                'sand shoal',
                'reef, buildup or bioherm',
                'perireef or subreef',
                'intrashelf/intraplatform reef',
                'platform/shelf-margin reef',
                'slope/ramp reef',
                'basin reef',
                'deep subtidal ramp',
                'deep subtidal shelf',
                'deep subtidal indet.',
                'offshore ramp',
                'offshore shelf',
                'offshore indet.',
                'slope',
                'basinal (carbonate)',
                'basinal (siliceous)',
                'marginal_marine_indet.',
                'coastal indet.',
                'estuary/bay',
                'lagoonal',
                'paralic indet.',
                'interdistributary bay',
                'delta front',
                'prodelta',
                'deltaic indet.',
                'foreshore',
                'shoreface',
                'transition zone/lower shoreface',
                'offshore',
                'submarine fan',
                'basinal (siliciclastic)',
                'deep-water indet.']

    ##### List and filename for features: landmass, mountain and icesheet

    terrestrial_list = ['terrestrial indet.',
                'fluvial indet.',
                'alluvial fan',
                'channel lag',
                'coarse channel fill',
                'fine channel fill',
                '"channel"',
                'wet floodplain',
                'dry floodplain',
                '"floodplain"',
                'crevasse splay',
                'levee',
                'mire/swamp',
                'fluvial-lacustrine_indet.',
                'delta plain',
                'fluvial-deltaic_indet.',
                'lacustrine - large',
                'lacustrine - small',
                'pond',
                'crater lake',
                'lacustrine_delta_plain',
                'lacustrine_interdistributary_bay',
                'lacustrine_delta_front',
                'lacustrine_pro-delta',
                'lacustrine_deltaic_indet.',
                'lacustrine indet.',
                'dune',
                'interdune',
                'loess',
                'eolian indet.',
                'cave',
                'fissure fill',
                'sinkhole',
                'karst indet.',
                'tar',
                'spring',
                'glacial']

    mountain_list = terrestrial_list

    icesheet_list = terrestrial_list

    ##### End

    ###################################################
    # define a function that will do the consistency check
    def DoPaleogeographyTest(reconstructed_point_features,Paleogeography_Filename,rotation_model,recon_time,environment_list,environment_label,env_num):

        PaleoGeog_Features = pygplates.FeatureCollection(Paleogeography_Filename)
        #pygplates.PlatePartitioner(): Partition features or geometries into plates.
        plate_partitioner = pygplates.PlatePartitioner(PaleoGeog_Features, rotation_model, reconstruction_time=recon_time) 

        tested_features = []
        untested_features = []

        for reconstructed_point in reconstructed_point_features:
            #plate_partitioner.partition_point(): if point is contained by any partitioning plates
            #reconstructed_point.get_reconstructed_geometry() means fossil points
            result = plate_partitioner.partition_point(reconstructed_point.get_reconstructed_geometry())
            if result is not None:

                # Get the environment for the point
                feature_environment = reconstructed_point.get_feature().get_shapefile_attribute('Environ')

                # See if the environment of the point matches the environment of the polygons 
                env_comparison = any(feature_environment in env for env in environment_list)
                reconstructed_point.get_feature().set_shapefile_attribute('GolEnv',environment_label)
                # Set the test indicator to have a value where the sign reflects the success of the test, 
                # while the absolute value depends on the environment
                if env_comparison is True:
                    reconstructed_point.get_feature().set_shapefile_attribute('Match',env_num)
                else:
                    reconstructed_point.get_feature().set_shapefile_attribute('Match',env_num*-1)
                reconstructed_point.get_feature().set_shapefile_attribute('PlateID',reconstructed_point.get_feature().get_reconstruction_plate_id())
                tested_features.append(reconstructed_point.get_feature())

            else:
                untested_features.append(reconstructed_point)

        # return reconstructed, tested features (with test results and plate ids assigned)
        return tested_features,untested_features

    ###################################################
    remaining_features = reconstructed_point_features

    if os.path.isfile(Icesheet_Filename):
        i_features,remaining_features = DoPaleogeographyTest(remaining_features,
                                          Icesheet_Filename,
                                          rotation_filename_for_paleogeog,
                                          recon_time,
                                          icesheet_list,
                                          'Icesheet',4)
        #print len(i_features),len(remaining_features)
    else:
        print 'No icesheet found'

    m_features,remaining_features = DoPaleogeographyTest(remaining_features,
                                      Mountain_Filename,
                                      rotation_filename_for_paleogeog,
                                      recon_time,
                                      mountain_list,
                                      'Mountain',3)
    #print len(m_features),len(remaining_features)

    lm_features,remaining_features = DoPaleogeographyTest(remaining_features,
                                       Landmass_Filename,
                                       rotation_filename_for_paleogeog,
                                       recon_time,
                                       terrestrial_list,
                                       'Landmass',2)
    #print len(lm_features),len(remaining_features)

    sm_features,remaining_features = DoPaleogeographyTest(remaining_features,
                                       ShallowMarine_Filename,
                                       rotation_filename_for_paleogeog,
                                       recon_time,
                                       shallow_marine_list,
                                       'ShallowMarine',1)
    #print len(sm_features),len(remaining_features)

    # Catch all the features that are not in any polygon
    unknown_features = []
    for reconstructed_point in reconstructed_point_features:
        if reconstructed_point.get_feature().get_shapefile_attribute('GolEnv')=='Unknown':
            reconstructed_point.get_feature().set_shapefile_attribute('PlateID',reconstructed_point.get_feature().get_reconstruction_plate_id())
            unknown_features.append(reconstructed_point.get_feature())
    #print len(unknown_features)

    ###################################################
    # Before saving to a single file, put together all the features into one collection
    all_features = []
    for feature in sm_features:
        all_features.append(feature)
    for feature in lm_features:
        all_features.append(feature)
    for feature in m_features:
        all_features.append(feature)

    if os.path.exists("i_features"):
        for feature in i_features:
            all_features.append(feature)
    else:
        print 'No icesheet found'

    for feature in unknown_features:
        all_features.append(feature)

    #print len(all_features)

    # Also before saving to file, assign the pbdb 'coarse' environment (based on the lookup table)
    # to each point, so that in GIS we can distinguish simply between 'Terrestrial' and'Marine' fossils
    for feature in all_features:
        # Get the environment for the point
        feature_environment = feature.get_shapefile_attribute('Environ')

        # See if the environment of the point matches the environment of the polygons 
        env_comparison = any(feature_environment in env for env in shallow_marine_list)
        if env_comparison is True:
            feature.set_shapefile_attribute('pbdbEnv','ShallowMarine')
        # See if the environment of the point matches the environment of the polygons 
        env_comparison = any(feature_environment in env for env in terrestrial_list)
        if env_comparison is True:
            feature.set_shapefile_attribute('pbdbEnv','Terrestrial')

    #Save fossil data as shapefile-format at present-day coordinates
    rotation_model_PresentDay = pygplates.RotationModel(rotation_filename_for_paleogeog)
    #EB2015v1 model, present-day paleogeog:
    OutputShapefileName_PresentDay = '/Users/wenchaocao/Research/7_Fossils_TestingPaleogeog/6_1_PresentDayFossils_TestingReconPaleogeog_Matthews2016_AfterGapsFixed_410_2Ma/PresentDayFossils_TestingReconPaleogeog_Matthews2016_AfterGapsFixed_'+str(recon_time)+'Ma.shp'

    pygplates.reconstruct(all_features,
                          rotation_model_PresentDay,
                          OutputShapefileName_PresentDay,
                          recon_time)

    #Save to file, all features lumped together and reconstructed using the rotation model
    #Previous tests do not output reconstructed fossils
    OutputShapefileName = '/Users/wenchaocao/Research/7_Fossils_TestingPaleogeog/6_2_ReconFossils_TestingReconPaleogeog_Matthews2016_AfterGapsFixed_410_2Ma/ReconFossils_TestingReconPaleogeog_Matthews2016_AfterGapsFixed_'+str(recon_time)+'Ma.shp'

    pygplates.reconstruct(all_features,
                          rotation_model,
                          OutputShapefileName,
                          recon_time)
    #print len(all_features)

    ###################################################
    #### Get a score of success or failure based on the pbdb fossil type (terrestrial or marine)
    print 'Time interval = %d' % recon_time
    print ''

    ShallowMarineTrue = 0
    ShallowMarineFalse = 0
    InvalidFossils = 0
    for feature in all_features:
        #print feature.get_shapefile_attribute('pbdbEnv')
        if feature.get_shapefile_attribute('pbdbEnv')=='ShallowMarine':
            if feature.get_shapefile_attribute('Match')>0:
                ShallowMarineTrue+=1
            elif feature.get_shapefile_attribute('Match')<0:
                ShallowMarineFalse+=1
            else:
                InvalidFossils+=1

    print 'PBDB Marine Fossils:'
    print 'Consistent = %d' % ShallowMarineTrue
    print 'Inconsistent = %d' % ShallowMarineFalse  
    print ''

    TerrestrialTrue = 0
    TerrestrialFalse = 0
    for feature in all_features:
        if feature.get_shapefile_attribute('pbdbEnv')=='Terrestrial':
            if feature.get_shapefile_attribute('Match')>0:
                TerrestrialTrue+=1
            elif feature.get_shapefile_attribute('Match')<0:
                TerrestrialFalse+=1
            else:
                InvalidFossils+=1

    print 'PBDB Terrestrial Fossils:'
    print 'Consistent = %d' % TerrestrialTrue
    print 'Inconsistent = %d' % TerrestrialFalse  
    print ''

    print 'InvalidFossils = %d'% InvalidFossils
    print ''

    print 'Number of tested fossils:'
    print ShallowMarineTrue + ShallowMarineFalse + TerrestrialTrue + TerrestrialFalse + InvalidFossils + len(partitioned_point_features_zero)
    print ''
    print 'Sum of Fossils extracted from PBDB:'
    print len(pbdb_subset)





### Test paleogeog after gaps fixed using ICS2016 time scale

In [4]:
import pygplates
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

%matplotlib inline


#########################################
###### Make a loop
Data_fossils = pd.read_excel('/Users/wenchaocao/Research/5_ConsistencyRatios_Fossils_Paleogeog/ReconPaleogeog_Matthews2016_AfterGapsFixed_TestByFossils_ICS2016/ReconPaleogeog_Matthews2016_AfterGapsFixed_TestByFossils_ICS2016.xlsx', delimiter=',')  
reconstruction_time_list_fossils = Data_fossils['Time intervals']
#FigNum_list_fossils = np.arange(64,17,-2)
FromAge_list_fossils = Data_fossils['From_age']
ToAge_list_fossils = Data_fossils['To_age']

recon_time = []
age_max = []
age_min = []

#########################################
###### Make a loop
Data_paleog = pd.read_csv('/Users/wenchaocao/Research/5_ConsistencyRatios_Fossils_Paleogeog/ReconPaleogeog_EB2015v1_AfterModifiedBasedOnFossils_410_2Ma/ReconPaleogeog_EB2015v1_AfterModifiedOnFossils_TestByFossils_410_2Ma.csv', delimiter=',')  
reconstruction_time_list_paleog = Data_paleog['Time intervals']
FigNum_list_paleog = np.arange(64,17,-2)
FromAge_list_paleog = Data_paleog['From_age']
ToAge_list_paleog = Data_paleog['To_age']

recon_time_paleog = []
FigNum_paleog = []
age_max_paleog = []
age_min_paleog = []

anchor_plate= 0

In [10]:
for i,recon_time,age_max,age_min,recon_time_paleog,FigNum_paleog,age_max_paleog,age_min_paleog in zip(np.arange(1,25,1),reconstruction_time_list_fossils,FromAge_list_fossils,ToAge_list_fossils,reconstruction_time_list_paleog,FigNum_list_paleog,FromAge_list_paleog,ToAge_list_paleog):
    print i,recon_time,age_max,age_min,recon_time_paleog,FigNum_paleog,age_max_paleog,age_min_paleog


    pbdb = pd.read_csv('/Users/wenchaocao/Research/6_CookieCutting_Corrections/Fossil_Collections_Genus_PBDB_2500_0Ma/2500Ma-collections.csv')

    #extract subsets
    pbdb_subset = pbdb[(pbdb.ma_max<=age_max) & (pbdb.ma_min>=age_min)] 

    # input
    base_directory = '/Users/wenchaocao/Research/6_CookieCutting_Corrections/'+str(recon_time_paleog)+'/7_2_Reconstructed_Paleogeog_Matthews2016_'+str(recon_time_paleog)+'Ma_GapsFixed/'
    ShallowMarine_Filename = '%s/sm_fig%d_%d_%d_reconstructed_%0.2fMa.shp' % (base_directory,FigNum_paleog,age_max_paleog,age_min_paleog,recon_time_paleog)
    Landmass_Filename = '%s/lm_fig%d_%d_%d_reconstructed_%0.2fMa.shp' % (base_directory,FigNum_paleog,age_max_paleog,age_min_paleog,recon_time_paleog)
    Mountain_Filename = '%s/m_fig%d_%d_%d_reconstructed_%0.2fMa.shp' % (base_directory,FigNum_paleog,age_max_paleog,age_min_paleog,recon_time_paleog)
    Icesheet_Filename = '%s/i_fig%d_%d_%d_reconstructed_%0.2fMa.shp' % (base_directory,FigNum_paleog,age_max_paleog,age_min_paleog,recon_time_paleog)


    # rotations:
    rotation_filename_for_fossils = ['/Users/wenchaocao/Research/6_CookieCutting_Corrections/Matthews++_410-0Ma_latest/Global_EB_250-0Ma_GK07_Matthews++.rot',
                                    '/Users/wenchaocao/Research/6_CookieCutting_Corrections/Matthews++_410-0Ma_latest/Global_EB_410-250Ma_GK07_Matthews++.rot']

    static_polygons = '/Users/wenchaocao/Research/6_CookieCutting_Corrections/Matthews++_410-0Ma_latest/StaticPolys/PresentDay_StaticPlatePolygons_Matthews++.shp'

    rotation_filename_for_paleogeog = '/Users/wenchaocao/Desktop/New_Rotation.rot'
    #######################################

    rotation_model = pygplates.RotationModel(rotation_filename_for_fossils)

    point_features = []

    for index,row in pbdb_subset.iterrows():
        point = pygplates.PointOnSphere(float(row.latdec),float(row.lngdec))
        #print row.LONG
        point_feature = pygplates.Feature()
        point_feature.set_geometry(point)
        point_feature.set_shapefile_attribute('Environ',str(row.environment))
        point_feature.set_shapefile_attribute('lithology1',str(row.lithology1))
        point_feature.set_shapefile_attribute('lithology2',str(row.lithology2))
        point_feature.set_shapefile_attribute('coll_no',str(row.collection_no))
        point_feature.set_shapefile_attribute('coll_auth',str(row['collection.authorizer']))
        point_feature.set_shapefile_attribute('coll_refno',str(row['collection.reference_no']))
        point_feature.set_shapefile_attribute('ma_max',row.ma_max)
        point_feature.set_shapefile_attribute('ma_min',row.ma_min)
        point_feature.set_shapefile_attribute('period',str(row.period))
        point_feature.set_shapefile_attribute('epoch',str(row.epoch))
        point_feature.set_shapefile_attribute('stage',str(row.stage))
        point_feature.set_shapefile_attribute('GolEnv','Unknown')
        point_feature.set_shapefile_attribute('pbdbEnv','Unknown')
        point_feature.set_shapefile_attribute('Match',0)
        point_feature.set_shapefile_attribute('FromAge',age_max) #added on 6/12/2016
        point_feature.set_shapefile_attribute('ToAge',age_min) #added on 6/12/2016
        point_feature.set_valid_time(age_max,age_min)
        point_features.append(point_feature)

    print len(point_features)

    # The partition points function can then be used as before
    partitioned_point_features = pygplates.partition_into_plates(static_polygons,
                                                           rotation_model,
                                                           point_features) 

    # If partitioned feature has plate id = 0, we throw it away 
    partitioned_point_features_not_zero = []
    partitioned_point_features_zero = []
    for point_feature in partitioned_point_features:
        if point_feature.get_reconstruction_plate_id() != 0:
            partitioned_point_features_not_zero.append(point_feature)
        else:
            partitioned_point_features_zero.append(point_feature)

    # Create a reconstructed version of the fossil features 

    reconstructed_point_features = []
    pygplates.reconstruct(partitioned_point_features_not_zero,
                          rotation_model,
                          reconstructed_point_features,
                          recon_time)
    #print len(partitioned_point_features_not_zero)
    #print len(partitioned_point_features_zero)

    ###################################################
    # This cell contains the mapping between paleo-environments descriptions in PBDB and our own system
    ##### List and filename for Shallow Marine

    shallow_marine_list = ['marine indet.',
                'carbonate indet.',
                'peritidal',
                'shallow subtidal indet.',
                'open shallow subtidal',
                'lagoonal/restricted shallow subtidal',
                'sand shoal',
                'reef, buildup or bioherm',
                'perireef or subreef',
                'intrashelf/intraplatform reef',
                'platform/shelf-margin reef',
                'slope/ramp reef',
                'basin reef',
                'deep subtidal ramp',
                'deep subtidal shelf',
                'deep subtidal indet.',
                'offshore ramp',
                'offshore shelf',
                'offshore indet.',
                'slope',
                'basinal (carbonate)',
                'basinal (siliceous)',
                'marginal_marine_indet.',
                'coastal indet.',
                'estuary/bay',
                'lagoonal',
                'paralic indet.',
                'interdistributary bay',
                'delta front',
                'prodelta',
                'deltaic indet.',
                'foreshore',
                'shoreface',
                'transition zone/lower shoreface',
                'offshore',
                'submarine fan',
                'basinal (siliciclastic)',
                'deep-water indet.']

    ##### List and filename for features: landmass, mountain and icesheet

    terrestrial_list = ['terrestrial indet.',
                'fluvial indet.',
                'alluvial fan',
                'channel lag',
                'coarse channel fill',
                'fine channel fill',
                '"channel"',
                'wet floodplain',
                'dry floodplain',
                '"floodplain"',
                'crevasse splay',
                'levee',
                'mire/swamp',
                'fluvial-lacustrine_indet.',
                'delta plain',
                'fluvial-deltaic_indet.',
                'lacustrine - large',
                'lacustrine - small',
                'pond',
                'crater lake',
                'lacustrine_delta_plain',
                'lacustrine_interdistributary_bay',
                'lacustrine_delta_front',
                'lacustrine_pro-delta',
                'lacustrine_deltaic_indet.',
                'lacustrine indet.',
                'dune',
                'interdune',
                'loess',
                'eolian indet.',
                'cave',
                'fissure fill',
                'sinkhole',
                'karst indet.',
                'tar',
                'spring',
                'glacial']

    mountain_list = terrestrial_list

    icesheet_list = terrestrial_list

    ##### End

    ###################################################
    # define a function that will do the consistency check
    def DoPaleogeographyTest(reconstructed_point_features,Paleogeography_Filename,rotation_model,recon_time,environment_list,environment_label,env_num):

        PaleoGeog_Features = pygplates.FeatureCollection(Paleogeography_Filename)
        #pygplates.PlatePartitioner(): Partition features or geometries into plates.
        plate_partitioner = pygplates.PlatePartitioner(PaleoGeog_Features, rotation_model, reconstruction_time=recon_time) 

        tested_features = []
        untested_features = []

        for reconstructed_point in reconstructed_point_features:
            #plate_partitioner.partition_point(): if point is contained by any partitioning plates
            #reconstructed_point.get_reconstructed_geometry() means fossil points
            result = plate_partitioner.partition_point(reconstructed_point.get_reconstructed_geometry())
            if result is not None:

                # Get the environment for the point
                feature_environment = reconstructed_point.get_feature().get_shapefile_attribute('Environ')

                # See if the environment of the point matches the environment of the polygons 
                env_comparison = any(feature_environment in env for env in environment_list)
                reconstructed_point.get_feature().set_shapefile_attribute('GolEnv',environment_label)
                # Set the test indicator to have a value where the sign reflects the success of the test, 
                # while the absolute value depends on the environment
                if env_comparison is True:
                    reconstructed_point.get_feature().set_shapefile_attribute('Match',env_num)
                else:
                    reconstructed_point.get_feature().set_shapefile_attribute('Match',env_num*-1)
                reconstructed_point.get_feature().set_shapefile_attribute('PlateID',reconstructed_point.get_feature().get_reconstruction_plate_id())
                tested_features.append(reconstructed_point.get_feature())

            else:
                untested_features.append(reconstructed_point)

        # return reconstructed, tested features (with test results and plate ids assigned)
        return tested_features,untested_features

    ###################################################
    remaining_features = reconstructed_point_features

    if os.path.isfile(Icesheet_Filename):
        i_features,remaining_features = DoPaleogeographyTest(remaining_features,
                                          Icesheet_Filename,
                                          rotation_filename_for_paleogeog,
                                          recon_time,
                                          icesheet_list,
                                          'Icesheet',4)
        #print len(i_features),len(remaining_features)
    else:
        print 'No icesheet found'

    m_features,remaining_features = DoPaleogeographyTest(remaining_features,
                                      Mountain_Filename,
                                      rotation_filename_for_paleogeog,
                                      recon_time,
                                      mountain_list,
                                      'Mountain',3)
    #print len(m_features),len(remaining_features)

    lm_features,remaining_features = DoPaleogeographyTest(remaining_features,
                                       Landmass_Filename,
                                       rotation_filename_for_paleogeog,
                                       recon_time,
                                       terrestrial_list,
                                       'Landmass',2)
    #print len(lm_features),len(remaining_features)

    sm_features,remaining_features = DoPaleogeographyTest(remaining_features,
                                       ShallowMarine_Filename,
                                       rotation_filename_for_paleogeog,
                                       recon_time,
                                       shallow_marine_list,
                                       'ShallowMarine',1)
    #print len(sm_features),len(remaining_features)

    # Catch all the features that are not in any polygon
    unknown_features = []
    for reconstructed_point in reconstructed_point_features:
        if reconstructed_point.get_feature().get_shapefile_attribute('GolEnv')=='Unknown':
            reconstructed_point.get_feature().set_shapefile_attribute('PlateID',reconstructed_point.get_feature().get_reconstruction_plate_id())
            unknown_features.append(reconstructed_point.get_feature())
    #print len(unknown_features)

    ###################################################
    # Before saving to a single file, put together all the features into one collection
    all_features = []
    for feature in sm_features:
        all_features.append(feature)
    for feature in lm_features:
        all_features.append(feature)
    for feature in m_features:
        all_features.append(feature)

    if os.path.exists("i_features"):
        for feature in i_features:
            all_features.append(feature)
    else:
        print 'No icesheet found'

    for feature in unknown_features:
        all_features.append(feature)

    #print len(all_features)

    # Also before saving to file, assign the pbdb 'coarse' environment (based on the lookup table)
    # to each point, so that in GIS we can distinguish simply between 'Terrestrial' and'Marine' fossils
    for feature in all_features:
        # Get the environment for the point
        feature_environment = feature.get_shapefile_attribute('Environ')

        # See if the environment of the point matches the environment of the polygons 
        env_comparison = any(feature_environment in env for env in shallow_marine_list)
        if env_comparison is True:
            feature.set_shapefile_attribute('pbdbEnv','ShallowMarine')
        # See if the environment of the point matches the environment of the polygons 
        env_comparison = any(feature_environment in env for env in terrestrial_list)
        if env_comparison is True:
            feature.set_shapefile_attribute('pbdbEnv','Terrestrial')

    #Save fossil data as shapefile-format at present-day coordinates
    rotation_model_PresentDay = pygplates.RotationModel(rotation_filename_for_paleogeog)
    #EB2015v1 model, present-day paleogeog:
    OutputShapefileName_PresentDay = '/Users/wenchaocao/Research/7_Fossils_TestingPaleogeog/6_1_PresentDayFossils_TestingReconPaleogeog_Matthews2016_AfterGapsFixed_410_2Ma/PresentDayFossils_TestingReconPaleogeog_Matthews2016_AfterGapsFixed_'+str(recon_time)+'Ma.shp'
    print OutputShapefileName_PresentDay

    pygplates.reconstruct(all_features,
                          rotation_model_PresentDay,
                          OutputShapefileName_PresentDay,
                          recon_time)

    #Save to file, all features lumped together and reconstructed using the rotation model
    #Previous tests do not output reconstructed fossils
    OutputShapefileName = '/Users/wenchaocao/Research/7_Fossils_TestingPaleogeog/6_2_ReconFossils_TestingReconPaleogeog_Matthews2016_AfterGapsFixed_410_2Ma/ReconFossils_TestingReconPaleogeog_Matthews2016_AfterGapsFixed_'+str(recon_time)+'Ma.shp'
    print OutputShapefileName
    
    pygplates.reconstruct(all_features,
                          rotation_model,
                          OutputShapefileName,
                          recon_time)
    #print len(all_features)

    ###################################################
    #### Get a score of success or failure based on the pbdb fossil type (terrestrial or marine)
    print 'Time interval = %0.2f' % recon_time
    print ''

    ShallowMarineTrue = 0
    ShallowMarineFalse = 0
    InvalidFossils = 0
    for feature in all_features:
        #print feature.get_shapefile_attribute('pbdbEnv')
        if feature.get_shapefile_attribute('pbdbEnv')=='ShallowMarine':
            if feature.get_shapefile_attribute('Match')>0:
                ShallowMarineTrue+=1
            elif feature.get_shapefile_attribute('Match')<0:
                ShallowMarineFalse+=1
            else:
                InvalidFossils+=1

    print 'PBDB Marine Fossils:'
    print 'Consistent = %0.2f' % ShallowMarineTrue
    print 'Inconsistent = %0.2f' % ShallowMarineFalse  
    print ''

    TerrestrialTrue = 0
    TerrestrialFalse = 0
    for feature in all_features:
        if feature.get_shapefile_attribute('pbdbEnv')=='Terrestrial':
            if feature.get_shapefile_attribute('Match')>0:
                TerrestrialTrue+=1
            elif feature.get_shapefile_attribute('Match')<0:
                TerrestrialFalse+=1
            else:
                InvalidFossils+=1

    print 'PBDB Terrestrial Fossils:'
    print 'Consistent = %0.2f' % TerrestrialTrue
    print 'Inconsistent = %0.2f' % TerrestrialFalse  
    print ''

    print 'InvalidFossils = %0.2f'% InvalidFossils
    print ''

    print 'Number of tested fossils:'
    print ShallowMarineTrue + ShallowMarineFalse + TerrestrialTrue + TerrestrialFalse + InvalidFossils + len(partitioned_point_features_zero)
    print ''
    print 'Sum of Fossils extracted from PBDB:'
    print len(pbdb_subset)

1 6.715 11.63 1.8 6 64 11 2
3775
No icesheet found
/Users/wenchaocao/Research/7_Fossils_TestingPaleogeog/6_1_PresentDayFossils_TestingReconPaleogeog_Matthews2016_AfterGapsFixed_410_2Ma/PresentDayFossils_TestingReconPaleogeog_Matthews2016_AfterGapsFixed_6.715Ma.shp
/Users/wenchaocao/Research/7_Fossils_TestingPaleogeog/6_2_ReconFossils_TestingReconPaleogeog_Matthews2016_AfterGapsFixed_410_2Ma/ReconFossils_TestingReconPaleogeog_Matthews2016_AfterGapsFixed_6.715Ma.shp
Time interval = 6.72

PBDB Marine Fossils:
Consistent = 1622.00
Inconsistent = 853.00

PBDB Terrestrial Fossils:
Consistent = 861.00
Inconsistent = 185.00

InvalidFossils = 253.00

Number of tested fossils:
3774

Sum of Fossils extracted from PBDB:
3775
2 16.035 20.44 11.63 14 62 20 11
1285
No icesheet found
/Users/wenchaocao/Research/7_Fossils_TestingPaleogeog/6_1_PresentDayFossils_TestingReconPaleogeog_Matthews2016_AfterGapsFixed_410_2Ma/PresentDayFossils_TestingReconPaleogeog_Matthews2016_AfterGapsFixed_16.035Ma.shp
/Users